In [0]:
# COMP5339 Week 11 Tutorial
# Material last updated: 14 May 2025
# Note materials were designed with the Roboto Condensed font, which can be installed here: https://www.1001fonts.com/roboto-condensed-font.html

from IPython.display import HTML
HTML('''
    <style> body {font-family: "Roboto Condensed Light", "Roboto Condensed";} h2 {padding: 10px 12px; background-color: #E64626; position: static; color: #ffffff; font-size: 40px;} .text_cell_render p { font-size: 15px; } .text_cell_render h1 { font-size: 30px; } h1 {padding: 10px 12px; background-color: #E64626; color: #ffffff font-size: 40px} .text_cell_render h3 { padding: 10px 12px; background-color: #0148A4; position: static; color: #ffffff; font-size: 20px;} h4:before{ 
    content: "@"; font-family:"Wingdings"; font-style:regular; margin-right: 4px;} .text_cell_render h4 {padding: 8px; font-family: "Roboto Condensed Light"; position: static; font-style: italic; background-color: #FFB800; color: #ffffff; font-size: 18px; text-align: center; border-radius: 5px;}input[type=submit] {background-color: #E64626; border: solid; border-color: #734036; color: white; padding: 8px 16px; text-decoration: none; margin: 4px 2px; cursor: pointer; border-radius: 20px;}</style>
    <script> code_show=true; function code_toggle() {if (code_show){$('div.input').hide();} else {$('div.input').show();} code_show = !code_show} $( document ).ready(code_toggle);</script>
    <form action="javascript:code_toggle()"><input type="submit" value="Hide/show all code."></form>
''')

# Week 11 - Image Data

Our final tutorial of new content revolves around one source of unstructured data - images. We'll be covering how images can be **represented, analysed, and segmented**, with an ultimate aim to extract **regions of interest**.

Download the [images.zip](https://unisyd-my.sharepoint.com/:u:/g/personal/lijun_chang_sydney_edu_au/EaEY-NoBFaxOqb6W2w3kNxoBCVVAJOFe8XIvtmb5-eIw8g?e=5Bp2fZ), put it under the same directory as the notebook, and unzip it.

Note this tutorial contains minimal "tasks" to be completed, it's intended more as a demonstration of what's possible.

## 1. Representing Images as Matrices

Images are ultimately comprised of many, *many* pixels, arranged in rows and columns. For example, an image with dimensions of 1920x1080 would imply 1920 pixel columns and 1080 pixel rows - more than 2 million pixels!

Let's start by first generating an image to see this for ourselves, before stepping it up with real images.

### 1.1 Simulated Images

These initially required modules will already be installed - `matplotlib` for producing visuals, `numpy` for handling the maths.

In [0]:
import matplotlib.pyplot as plt
import numpy as np

To demonstrate what can be done with **matrices**, we can generate an array with 10 columns and 10 rows using `numpy`. This will be an n-dimensional array, where each item is a randomly generated decimal between 0 and 1.

In [0]:
rand = np.random.random([10, 10])
print(f'Type: {type(rand)}\nData type: {rand.dtype}\nSize: {rand.shape}')

rand

This can effectively be interpreted as a 10x10 pixel **grayscale** image, where numbers closer to 0 are darker, and numbers closer to 1 are lighter.

Note the cell below utilises a *semicolon* (;), which simply suppresses additional text output. This can instead be done by adding a `plt.show()` line at the end.

In [0]:
plt.imshow(rand, cmap='gray');

Since it is effectively just a matrix, we can **re-define** individual pixels at will. For example, let's set the top-left pixel to be completely white, by accessing it at row 0, column 0, and setting its value to 1.

In [0]:
rand[0,0] = 1
plt.imshow(rand, cmap='gray');

### 1.2 Real Images

Let's take this up a notch by interpreting real, colour images as a matrix, and investigating the transformations we can apply there. For this, we'll import the 'images' submodule within `matplotlib`, and read in the 'sample.png' image, which should be produced in the same repository as this notebook when the provided `images.zip` archive is extracted.

In [0]:
import matplotlib.image as mpimg
img = mpimg.imread('images/sample.png')
plt.imshow(img);

Great, `matplotlib` can not only generate nice statistical graphs, but memes as well.

So how is this image being stored? Let's first analyse the structure, using the `.shape` attribute.

In [0]:
img.shape

This indicates that the image contains 1080 pixel rows, 1920 pixel columns, and then 4 attributes for each of the pixels. To investigate the 4 attributes of each pixel, we'll return just the top-left pixel (row 0, column 0).

In [0]:
img[0, 0]

As explained in the [docs](https://au.mathworks.com/help/matlab/ref/imread.html#btnczv9-1-transparency), the first three numbers here are **[RGB elements](https://en.wikipedia.org/wiki/RGB_color_model)** - the combination of red, green and blue required to produce that particular colour. The fourth indicates the transparency, since the image type is a `.png`, which is capable of producing transparent backgrounds. In our case, since the whole image is fully populated, this will be '1' for all pixels.

Traditionally in computing, RGB elements are provided as integers out of 256. If we ignore the transparency number, multiply by 256, and round to the nearest number, we can retrieve these values.

In [0]:
(img[0, 0, :3]*256).astype(int)

The following cell is slightly mathsy, but converts these numbers from an RGB triplet into the universal **[six-character hexadecimal format](https://en.wikipedia.org/wiki/Web_colors)**. Try Googling the result of this cell, and confirm that the colour looks to be the same as what is produced in the top left of our original image (should both be a shade of blue!).

In [0]:
def hex_from_rgb(imgmatrix, r=0, c=0):
  return '#%02x%02x%02x' % tuple((imgmatrix[r, c, :3]*256).astype(int))

print(hex_from_rgb(img))

## 2. Image Analysis

Many of the following examples in this section are based on [the matplotlib documentation](https://matplotlib.org/3.5.0/tutorials/introductory/images.html).

It's great to be able to store image data and understand its components, but we're likely interested in conducting transformations, understanding features of the image, or extracting components.

### 2.1 Luminosity

Now we know the `shape` of an image's matrix includes the number of rows, number of columns, and then number of colour features, we can convert any image to a **grayscale** (luminosity) image by zero-ing out the last dimension. If we do so here, the results can be plotted using a gray colour map, to produce a black and white image.

In [0]:
lum_img = img[:, :, 0]
plt.imshow(lum_img, cmap="gray")
plt.colorbar();

We've only used gray so far due to its ease of interpretability with the 0-to-1 scale, but there is numerous alternative **[colour maps](https://matplotlib.org/3.5.0/tutorials/colors/colormaps.html)** we can use. The below selection of colour palettes produces art perhaps akin to [Andy Warhol's](https://en.wikipedia.org/wiki/Shot_Marilyns) work.

In [0]:
%matplotlib inline
fig = plt.figure(figsize=(10, 7))
for i, color in enumerate(['viridis', 'hot', 'spring', 'ocean']):
    fig.add_subplot(2, 2, i+1)
    plt.imshow(lum_img, cmap=color)

### 2.2 Segmentation

Moving towards region extraction now, we can analyse the **frequency of pixels in a given colour range**. These histograms can be useful to determine what thresholds would be required for retaining only a subset of pixels, to examine a specific data range, enhance image contrast, or expand the contrast.

In [0]:
plt.hist(lum_img.ravel(), bins=256, range=(0.0, 1.0), fc='k');

The histogram indicates two peaks - one around grayscale colour values of 0.2, and the other around 0.8. If we were to only retain pixels from the lower range, for example 0 to 0.3, the resulting image would only contain **darker** pixels:

In [0]:
plt.imshow(lum_img, clim=(0, 0.3), cmap="gray");

Let's produce a side-by-side comparison if we were to try and **remove the background** with a simple threshold, and only retain the relatively darker pixels. The below code sets the pixel brightness limit at 0.75, thereby removing most of the background - not a perfect system given it hasn't entirely retained the character's features, but a decent start.

In [0]:
# original image
fig = plt.figure()
ax = fig.add_subplot(1, 2, 1)
imgplot = plt.imshow(lum_img, cmap="gray")
ax.set_title('Before')
plt.colorbar(orientation='horizontal')

# only darker pixels
ax = fig.add_subplot(1, 2, 2)
imgplot = plt.imshow(lum_img, cmap="gray")
imgplot.set_clim(0.0, 0.75)
ax.set_title('After')
plt.colorbar(orientation='horizontal');

### 2.3 Interpolation

More complex image operations can be achieved with the `Pillow` library. Install this using a command like `pip install pillow` in your terminal.

One such operation includes resizing images to different resolutions. For example, we can produce a more pixelated, lower-quality 64x64 image, like below (which resizes the image in-place):

In [0]:
from PIL import Image
img = Image.open('images/sample.png')
img.thumbnail((64, 64), Image.BOX)
plt.imshow(img);

"Smoothing" out pixelated images can be done with the **interpolation** argument, which employs a mathematical formula of your choice (in this case "gaussian", but there's many others in the [docs](https://matplotlib.org/3.5.0/api/_as_gen/matplotlib.pyplot.imshow.html)).

In [0]:
plt.imshow(img, interpolation="gaussian");

### 2.4 Export

The `skimage` (scikit-image) module also contains a number of handy operations for dealing with images. For example, should we wish to convert the image format from its current state (PNG) to another (e.g. TIFF), it's `io.imsave` function can handle this.

In [0]:
from skimage import io
inputimage = io.imread('images/sample.png')
io.imsave('sample.tiff', inputimage)
print('Saved as a TIFF file.')

Converting from PNG to JPG requires a bit more work, since PNG can handle transparency, but JPG cannot. A conversion workaround:

In [0]:
Image.open('images/sample.png').convert('RGB').save('sample.jpg', 'JPEG')

To confirm this was successful, let's load back in the JPG image we've just produced. Should be a familiar face!

In [0]:
img = Image.open('sample.jpg')
imgplot = plt.imshow(img)

# 3. ROI Extraction

Applying this to a real world example - additionally contained within the `images.zip` we expanded earlier are two folders, 'treated' and 'untreated'. These contain **real DNA images** captured by microscopes at the Children’s Medical Research Institute, where researchers study **drug efficacy for killing cancer cells**. Researchers take snapshots of DNA before applying any drug (the 'untreated' images), and then take another set of images after applying the drug (the 'treated' group).

### 3.1 Import

Let's investigate the first untreated example, reading it in with `skimage`. From there, we'll check its dimensions and datatypes:

In [0]:
dna = io.imread("images/untreated/1.tif")
print(f'Type: {type(dna)}\nData type: {dna.dtype}\nSize: {dna.shape}')

plt.imshow(dna, cmap='gray');

The DNA within the image is clearly the brighter item in the centre, but there's some background noise nearer the top of the image we'll likely wish to eliminate. We'll again investigate the **histogram** to see if a global threshold can be used to remove the background.

In [0]:
plt.hist(dna.ravel(), bins=256, fc='k');

### 3.2 Thresholding

Most of the values appear to be under the 2000 mark. If we only retain the pixels with values below 2500, we can visualise which portion of the image that will retain. **Try adjusting** the value below to different thresholds to see how the mask will change:

In [0]:
print(dna)
plt.imshow(dna>2500, cmap='gray');

Until now, we've only attempted a *global* threshold - a fixed value that determines which pixels are retained, and which are not. More complex thresholding algorithms are available in the `skimage.filters` module, and can be handily experimented with using the `try_all_threshold()` function ([docs](https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.try_all_threshold) here). **Which seems to be the most successful?**

In [0]:
from skimage.filters import try_all_threshold
fig, ax = try_all_threshold(dna, figsize=(10, 8), verbose=False);

Once we've selected a technique above, we can apply it individually. For example, using the `otsu` technique above:

In [0]:
from skimage.filters import threshold_otsu
dna_bw = dna > threshold_otsu(dna)
plt.imshow(dna_bw, cmap='gray');

Another technique that can be employed involves **removing smaller objects** detected in the image. This can be done using the `morphology.remove_small_objects()` function ([docs](https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.remove_small_objects) here), and can be quite effective in these scenarios:

In [0]:
from skimage import morphology
dna_bw = morphology.remove_small_objects(dna_bw)
plt.imshow(dna_bw, cmap='gray');

Reminder that the visuals above are simply image **masks** - they indicate *which* pixels will be retained, not the final picture. If we create a copy of the original image, and set all pixels not included in the mask to 0, the resulting image will be a much cleaner snapshot of the DNA snippet we're interested in.

In [0]:
dna_clean = dna
dna_clean[~dna_bw]=0
plt.imshow(dna_clean, cmap='gray');

### 3.3 Metrics

With background noise removed, we can begin calculating some metrics of the resulting area. Again using `skimage`, we can record the size of the DNA using techniques such as measuring its **perimeter**. Note the 'neighbourhood' argument, which is used for border pixel determination, and can be adjusted, yielding slightly different results.

In [0]:
import skimage.measure
skimage.measure.perimeter(dna_bw, neighborhood=4)

Perimeter is just one metric possible - using the `regionprops()` function ([docs](https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprops) here), we can use many more:

In [0]:
dna_props = skimage.measure.regionprops(skimage.measure.label(dna_bw), dna)
print(f"""Area: {dna_props[0].area}
Perimeter: {dna_props[0].perimeter}
Intensity: {dna_props[0].mean_intensity}""")

Others available:

In [0]:
print([prop for prop in dna_props[0]])

### 3.4 Analysis

Now that we've experimented with these base functionalities, we can scale up to all images in the directory. The function defined below uses the `otsu` thresholding technique and `remove_small_objects()` function from above to **clean the image, before measuring** the area, perimeter and mean intensity of the DNA in each image. Simply provide a filepath to an image to be read, and a list will be retained containing the image title, and the three metrics.

In [0]:
def processimages(infile):
    dna = io.imread(infile)
    dna_bw = dna > threshold_otsu(dna)
    dna_bw = morphology.remove_small_objects(dna_bw)
    
    dna_props = skimage.measure.regionprops(skimage.measure.label(dna_bw), dna)
    
    area = dna_props[0].area
    perimeter = dna_props[0].perimeter
    mean_intensity = dna_props[0].mean_intensity

    return [infile.split('/')[-1].split('.')[0], area, perimeter, mean_intensity]

processimages("images/untreated/1.tif")

Let's apply this to **every image** in the 'untreated' directory, saving the results as an 'untreated' list of lists, and then do the exact same for the 'treated' images (should only take a few seconds!).

In [0]:
import os
untreated = []
filepath = 'images/untreated/'
for img in [x for x in os.listdir(filepath) if x.endswith('.tif')]:
    untreated.append(processimages(filepath+img))
print('Untreated processing complete.')

treated = []
filepath = 'images/treated/'
for img in [x for x in os.listdir(filepath) if x.endswith('.tif')]:
    treated.append(processimages(filepath+img))
print('Treated processing complete.')

So does the distribution of the area of the observed DNA strands differ significantly between pre-treatment and post-treatment? **What could be concluded from the below visual** (acknowledging of course that this is a relatively small subset)?

In [0]:
plt.boxplot([[x[1] for x in untreated], [x[1] for x in treated]], vert=False, notch=True)
plt.yticks([1,2], ['Untreated DNA', 'Treated DNA'])
plt.xlabel('Area')
plt.title('Untreated vs Treated Area of DNA')
plt.show()